In [0]:
from statsmodels.tsa.stattools import adfuller
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Helper Function

In [0]:
def getScaledTrainTextDataset(dataset_scaled, trainRate=0.3):
    print('Shape: ',dataset_scaled.shape)
    train_size = int(len(dataset_scaled)*trainRate)
    test_size = len(dataset_scaled) - train_size
    print('Trainsize:',train_size, ' - Testsize:',test_size)
    train_start_index,train_end_index = 0,train_size
    test_start_index,test_end_index = train_size,len(dataset_scaled)
    train = dataset_scaled[0:train_size]
    test = dataset_scaled[train_size:] 
    return train,test

# convert an array of values into a dataset matrix
def create_XY_lookback_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back)]
		dataX.append(a)
		dataY.append(dataset[i + look_back])
	return numpy.array(dataX), numpy.array(dataY)

def create_XY_lookback_dataset_multistepOutput(dataset, look_back=1, look_forward = 1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1-look_forward):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back:i+look_back+look_forward, 0])
	return numpy.array(dataX), numpy.array(dataY)

def calculateRMSE(testPredict,trainY,testY,inverse_transform=True, verbose= 1):
    
    if inverse_transform:
        testPredict_inverse = scaler.inverse_transform(testPredict)
        testY_inverse = scaler.inverse_transform([testY])  
    else:
        testPredict_inverse = testPredict
        testY_inverse = testY
        
    # calculate root mean squared error
    testScore = math.sqrt(mean_squared_error(testY_inverse[0], testPredict_inverse[:,0]))
    if verbose == 1:
        print('Test Score: %.2f RMSE' % (testScore))
        print('Persistent Model Testscore small:',global_testPredict_small, ' - Persistent Model Testscore big:', global_testPredict_big)
        
    return  testScore

def calculateRMSE_MultipleOutput(trainPredict,testPredict,trainY,testY,inverse_transform=True, verbose= 1):
    
    if inverse_transform:
        trainPredict_inverse = scaler.inverse_transform(trainPredict)
        trainY_inverse = scaler.inverse_transform(trainY)
        testPredict_inverse = scaler.inverse_transform(testPredict)
        testY_inverse = scaler.inverse_transform(testY)  
    else:
        trainPredict_inverse = trainPredict
        trainY_inverse = trainY
        testPredict_inverse = testPredict
        testY_inverse = testY
        
    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY_inverse[0], trainPredict_inverse[:,0]))
    testScore = math.sqrt(mean_squared_error(testY_inverse[0], testPredict_inverse[:,0]))
    if verbose == 1:
        print('Train Score: %.2f RMSE' % (trainScore))
        print('Test Score: %.2f RMSE' % (testScore))
        print('Persistent Model Testscore small:',global_testPredict_small, ' - Persistent Model Testscore big:', global_testPredict_big)
        
    return trainScore, testScore

def plotErrorPrediction(dataset_scaled, testPredict, trainPredict, ShowTestError=True, inverse_transform=True):
    if inverse_transform:
        trainPredict_inverse = scaler.inverse_transform(trainPredict)
        testPredict_inverse = scaler.inverse_transform(testPredict)
    else:
        trainPredict_inverse = trainPredict
        testPredict_inverse = testPredict

    # shift train predictions for plotting
    trainPredictPlot = numpy.zeros_like(dataset_scaled)
    trainPredictPlot[:, :] = numpy.nan
    trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict_inverse
    # shift test predictions for plotting
    testPredictPlot = numpy.empty_like(dataset_scaled)
    testPredictPlot[:, :] = numpy.nan
    testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict_inverse    
    
    error_test = dataset-testPredictPlot
    error_test[np.isnan(error_test)] = 0.
    # error_test[error_training<2000] = 0.
    error_test = np.abs(error_test)
    import seaborn as sns
    sns.distplot(error_test[error_test!=0])
    print(pandas.DataFrame(error_test[error_test!=0]).describe())
    return error_test, trainPredictPlot,testPredictPlot

def plotErrorPredictionValidation(dataset_scaled, validationPredict, testPredict, trainPredict, ShowTestError=True, inverse_transform=True):
    if inverse_transform:
        trainPredict_inverse = scaler.inverse_transform(trainPredict)
        testPredict_inverse = scaler.inverse_transform(testPredict)
    else:
        trainPredict_inverse = trainPredict
        testPredict_inverse = testPredict

    # shift train predictions for plotting
    trainPredictPlot = numpy.zeros_like(dataset_scaled)
    trainPredictPlot[:, :] = numpy.nan
    trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict_inverse
    # shift test predictions for plotting
    testPredictPlot = numpy.empty_like(dataset_scaled)
    testPredictPlot[:, :] = numpy.nan
    testPredictPlot[len(trainPredict)+(look_back*2)+1+len(validationPredict):len(dataset)-1, :] = testPredict_inverse    
    
    error_test = dataset-testPredictPlot
    error_test[np.isnan(error_test)] = 0.
    # error_test[error_training<2000] = 0.
    error_test = np.abs(error_test)
    import seaborn as sns
    sns.distplot(error_test[error_test!=0])
    print(pandas.DataFrame(error_test[error_test!=0]).describe())
    return error_test, trainPredictPlot,testPredictPlot

# OCSVM

In [0]:
import warnings
from sklearn.cluster import KMeans
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_squared_error
from math import sqrt
from pandas import read_csv
import pandas as pd
from pandas import DataFrame
from pandas import concat
import numpy as np 
from matplotlib import pyplot
from sklearn.svm import OneClassSVM
from sklearn import preprocessing
import sys

class OneClassSVM_AnomalyDetection:
    def __init__(self,path, window_width, nu, train_rate):
        self.df = read_csv(path, header=0, index_col=0, parse_dates=True,squeeze=True)
        self.df = self.df.reset_index(drop=True)
        self.df.rename(columns={'anomaly':'is_anomaly'}, inplace=True)
        self.nu = nu
        self.window_width = window_width
        series = pd.DataFrame(self.df.iloc[:,0].values)  
        self.values = DataFrame(series.values)
        self.dataframe = concat([self.values.shift(1), self.values], axis=1)
        self.dataframe.columns = ['t', 't+1']

        self.train_size = int(len(self.values) * train_rate)

        # train_labeled, test_labeled = self.dataframe.values[1:self.train_size], self.dataframe.values[self.train_size:]
        # self.train_X, self.train_y = train_labeled[:,0], train_labeled[:,1]
        # self.test_X, self.test_y = test_labeled[:,0], test_labeled[:,1]     
        # self.create_persistence()

        # X = series.values
        # self.train, self.test = X[1:self.train_size], X[self.train_size:]    

    def __build_sets(self):
        train_labeled, test_labeled = self.dataframe.values[1:self.train_size], self.dataframe.values[self.train_size:]
        self.train_X, self.train_y = train_labeled[:,0], train_labeled[:,1]
        self.test_X, self.test_y = test_labeled[:,0], test_labeled[:,1]   

        X = self.dataframe.iloc[:,1].values
        self.train, self.test = X[1:self.train_size], X[self.train_size:]    

    def standardize_dataframe(self):
        X = self.dataframe.values
        self.scalar = preprocessing.StandardScaler().fit(X)
        X = self.scalar.transform(X)
        self.dataframe = pd.DataFrame(X)

    def inverse_standardize_dataframe(self):
        X = self.dataframe.values
        X = self.scalar.inverse_transform(X)
        self.dataframe = pd.DataFrame(X)



    def model_persistence(self, x):
        return x
        
    def create_persistence(self):
        rmse = sqrt(mean_squared_error(self.dataframe['t'].iloc[self.train_size:], self.dataframe['t+1'].iloc[self.train_size::]))
#         print('Persistent Model RMSE: %.3f' % rmse)   

    def fit(self):
        self.create_persistence()
        self.standardize_dataframe()
        self.__build_sets()
                
        self.compute_anomalyScores()
        self.inverse_standardize_dataframe()
    
    def getWindowedVectors(self, X):
        vectors = []
        for i,_ in enumerate(X[:-self.window_width+1]):
            vectors.append(X[i:i+self.window_width])
        return vectors

    def compute_anomalyScores(self):
        self.errors = np.zeros_like(self.test)
        # compute anomalies
        warnings.filterwarnings("ignore")

        # history = self.getWindowedVectors(self.train)

        for i,_ in enumerate(self.test[:-self.window_width+1]):
            sys.stdout.write('\r'+str(i)+':'+str(len(self.test) - self.window_width))

            window = self.test[i:i+self.window_width]
            window2D = np.zeros((len(window),2))
            window2D[:,1] = window
            clf=OneClassSVM(nu=self.nu)
            clf.fit(window2D)
            error = clf.decision_function(window2D) 
            error[error>0] = 0
            self.errors[i:i+self.window_width] += error*-10


        # normalize anomaly score
        self.errors[:-self.window_width+1] /= self.window_width
        for i,error in enumerate(self.test[-self.window_width+1:]):
            self.errors[-self.window_width + 1 + i] /=self.window_width-(i+1)

        # self.errors_original = self.errors
        # scalar = preprocessing.MinMaxScaler((0,1)).fit(self.errors.reshape(-1,1))
        # self.errors = scalar.transform(self.errors.reshape(-1,1))*10


    def plot(self):
        # plot predicted error
        pyplot.figure(figsize=(50,5))
        pyplot.plot(self.test)
        # pyplot.plot(self.predictions, color='red')
        pyplot.plot(self.errors, color = 'red',  linewidth=0.5)
        pyplot.show()

    def get_roc_auc(self, plot=True, verbose=True):
        # get the predicted errors of the anomaly points
        indices = self.df[self.df['is_anomaly']==1].index >self.train_size
        true_anomaly_predicted_errors = self.errors[self.df[self.df['is_anomaly']==1].index[indices] - self.train_size ]
        if len(true_anomaly_predicted_errors) == 0:
            return np.nan
        # sort them 
        true_anomaly_predicted_errors = np.sort(true_anomaly_predicted_errors,axis=0).reshape(-1)
        true_anomaly_predicted_errors_extended = np.r_[np.linspace(0,true_anomaly_predicted_errors[0],40)[:-1],true_anomaly_predicted_errors]
        true_anomaly_predicted_errors_extended = np.r_[true_anomaly_predicted_errors_extended, true_anomaly_predicted_errors_extended[-1] + np.mean(true_anomaly_predicted_errors_extended)]
                # now iterate thru the predicted errors from small to big
        # for each value look how much other points have equal or bigger error
        FPR = [] # fp/n  https://en.wikipedia.org/wiki/Sensitivity_and_specificity
        TPR = [] # tp/p
        p = len(true_anomaly_predicted_errors)
        Thresholds = []
        for predictederror in true_anomaly_predicted_errors_extended:
            threshold = predictederror
            tp = len(true_anomaly_predicted_errors[true_anomaly_predicted_errors>= threshold])
            fp = len(self.errors[self.errors>=threshold])-len(true_anomaly_predicted_errors[true_anomaly_predicted_errors>=threshold])
            
            fpr =fp/len(self.errors)
            FPR.append(fpr)
            TPR.append(tp/p)
            if verbose:
                print("Threshold: {0:25}  - FP: {1:4} - TP: {2:4} - FPR: {3:21} - TPR: {4:4}".format(threshold,fp, tp, fpr, tp/p))

        import matplotlib.pyplot as plt
        if plot:
            plt.figure()
            plt.axis([0, 1, 0, 1])
            plt.plot(FPR,TPR)
            plt.show() 

        # This is the AUC
        from sklearn.metrics import auc
        print('AUC: ' ,auc(FPR,TPR)        )
        return auc(FPR,TPR)
    
# iforest = OneClassSVM_AnomalyDetection('Univariate/YahooServiceNetworkTraffic/A1Benchmark/real_1.csv',30,0.7,0.3)
# iforest.fit()
# iforest.plot()
# iforest.get_roc_auc(verbose=False)

## Evaluation

### Results of NYCT

In [0]:

import os
import datetime

startTime = datetime.datetime.now()

import glob


cnn = OneClassSVM_AnomalyDetection( 'drive/My Drive/MT/Experiments/Univariate/NYC_Taxi/nyc_taxi.csv', 40,0.1,0.3)
cnn.fit()
cnn.get_roc_auc(plot=False,verbose=False)

        
endTime = datetime.datetime.now()
diff = endTime - startTime
print('Time: ',diff)
cnn = OneClassSVM_AnomalyDetection( 'drive/My Drive/MT/Experiments/Univariate/NYC_Taxi/nyc_taxi.csv', 40,0.1,0.3)

startTime = datetime.datetime.now()
cnn.fit()       
endTime = datetime.datetime.now()
diff = endTime - startTime
print('Inference Time: ',diff)

7184:7184AUC:  0.5210562102565175
Time:  0:00:06.942960
7184:7184Inference Time:  0:00:07.040107


# XGBoost

In [0]:
import warnings
from sklearn.cluster import KMeans
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_squared_error
from math import sqrt
from pandas import read_csv
import pandas as pd
from pandas import DataFrame
from pandas import concat
import numpy as np 
from matplotlib import pyplot
from xgboost import XGBRegressor
from sklearn import preprocessing
import sys

class XGBRegressor_AnomalyDetection:
    def __init__(self,path, window_width, nu, train_rate):
        self.df = read_csv(path, header=0, index_col=0, parse_dates=True,squeeze=True)
        self.df = self.df.reset_index(drop=True)
        self.df.rename(columns={'anomaly':'is_anomaly'}, inplace=True)
        self.nu = nu
        self.window_width = window_width
        series = pd.DataFrame(self.df.iloc[:,0].values)  
        self.values = DataFrame(series.values)
        self.dataframe = concat([self.values.shift(1), self.values], axis=1)
        self.dataframe.columns = ['t', 't+1']

        self.train_size = int(len(self.values) * train_rate)

        # train_labeled, test_labeled = self.dataframe.values[1:self.train_size], self.dataframe.values[self.train_size:]
        # self.train_X, self.train_y = train_labeled[:,0], train_labeled[:,1]
        # self.test_X, self.test_y = test_labeled[:,0], test_labeled[:,1]     
        # self.create_persistence()

        # X = series.values
        # self.train, self.test = X[1:self.train_size], X[self.train_size:]    

    def __build_sets(self):
        train_labeled, test_labeled = self.dataframe.values[1:self.train_size], self.dataframe.values[self.train_size:]
        self.train_X, self.train_y = train_labeled[:,0], train_labeled[:,1]
        self.test_X, self.test_y = test_labeled[:,0], test_labeled[:,1]   

        X = self.dataframe.iloc[:,1].values
        self.train, self.test = X[1:self.train_size], X[self.train_size:]    

    def standardize_dataframe(self):
        X = self.dataframe.values
        self.scalar = preprocessing.StandardScaler().fit(X)
        X = self.scalar.transform(X)
        self.dataframe = pd.DataFrame(X)

    def inverse_standardize_dataframe(self):
        X = self.dataframe.values
        X = self.scalar.inverse_transform(X)
        self.dataframe = pd.DataFrame(X)



    def model_persistence(self, x):
        return x
        
    def create_persistence(self):
        rmse = sqrt(mean_squared_error(self.dataframe['t'].iloc[self.train_size:], self.dataframe['t+1'].iloc[self.train_size::]))
#         print('Persistent Model RMSE: %.3f' % rmse)   

    def fit(self):
        self.create_persistence()
        self.__build_sets()
                
        self.compute_anomalyScores()
    
    def getWindowedVectors(self, X):
        vectors = []
        for i,_ in enumerate(X[:-self.window_width+1]):
            vectors.append(X[i:i+self.window_width])
        return vectors

    def compute_anomalyScores(self):

        self.xgb = XGBRegressor()
        self.xgb.fit(self.train_X.reshape(-1,1),self.train_y.reshape(-1,1))

        self.predictions = self.xgb.predict(self.test_X.reshape(-1,1))
        rmse = sqrt(mean_squared_error(self.test, self.predictions))
        self.errors = np.absolute(self.test - np.array(self.predictions))
#         print('Prediction Test RMSE: %.3f' % rmse)
    

    def plot(self):
        # plot predicted error
        pyplot.figure(figsize=(50,5))
        pyplot.plot(self.test)
        pyplot.plot(self.predictions, color='blue')
        pyplot.plot(self.errors, color = 'red',  linewidth=0.5)
        pyplot.show()

    def get_roc_auc(self, plot=True, verbose=True):
        # get the predicted errors of the anomaly points
        indices = self.df[self.df['is_anomaly']==1].index >self.train_size
        true_anomaly_predicted_errors = self.errors[self.df[self.df['is_anomaly']==1].index[indices] - self.train_size ]
        if len(true_anomaly_predicted_errors) == 0:
            return np.nan
        # sort them 
        true_anomaly_predicted_errors = np.sort(true_anomaly_predicted_errors,axis=0).reshape(-1)
        true_anomaly_predicted_errors_extended = np.r_[np.linspace(0,true_anomaly_predicted_errors[0],40)[:-1],true_anomaly_predicted_errors]
        true_anomaly_predicted_errors_extended = np.r_[true_anomaly_predicted_errors_extended, true_anomaly_predicted_errors_extended[-1] + np.mean(true_anomaly_predicted_errors_extended)]
                # now iterate thru the predicted errors from small to big
        # for each value look how much other points have equal or bigger error
        FPR = [] # fp/n  https://en.wikipedia.org/wiki/Sensitivity_and_specificity
        TPR = [] # tp/p
        p = len(true_anomaly_predicted_errors)
        Thresholds = []
        for predictederror in true_anomaly_predicted_errors_extended:
            threshold = predictederror
            tp = len(true_anomaly_predicted_errors[true_anomaly_predicted_errors>= threshold])
            fp = len(self.errors[self.errors>=threshold])-len(true_anomaly_predicted_errors[true_anomaly_predicted_errors>=threshold])
            
            fpr =fp/len(self.errors)
            FPR.append(fpr)
            TPR.append(tp/p)
            if verbose:
                print("Threshold: {0:25}  - FP: {1:4} - TP: {2:4} - FPR: {3:21} - TPR: {4:4}".format(threshold,fp, tp, fpr, tp/p))

        import matplotlib.pyplot as plt
        if plot:
            plt.figure()
            plt.axis([0, 1, 0, 1])
            plt.plot(FPR,TPR)
            plt.show() 

        # This is the AUC
        from sklearn.metrics import auc
        print('AUC: ' ,auc(FPR,TPR)        )
        return auc(FPR,TPR)


## Evaluation

### Results of NYCT

In [0]:

import os
import datetime

startTime = datetime.datetime.now()

import glob


cnn = XGBRegressor_AnomalyDetection( 'drive/My Drive/MT/Experiments/Univariate/NYC_Taxi/nyc_taxi.csv', 1000,0.5,0.3)
cnn.fit()
cnn.get_roc_auc(plot=False,verbose=False)

        
endTime = datetime.datetime.now()
diff = endTime - startTime
print('Time: ',diff)

startTime = datetime.datetime.now()
cnn.xgb.predict(cnn.test_X.reshape(-1,1))
endTime = datetime.datetime.now()
diff = endTime - startTime
print('Inference Time: ',diff)



[03:44:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
AUC:  0.4602617410866643
Time:  0:00:00.139282
Inference Time:  0:00:00.020563
